In [329]:
""" In this notebook I use GridSearch and RandomSearch for tuning hyperparams of CNN-model """

import os
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras as K
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from keras.utils import np_utils

In [167]:
def prepare_data(dataset='mnist'):
    if dataset.lower() == 'mnist':
        (x_train, y_train), (x_test, y_test) = K.datasets.mnist.load_data()

        x_train = x_train.astype("float32")
        x_test = x_test.astype("float32")

        x_train /= 255
        x_test /= 255

        x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
        x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))
        
        y_train = np_utils.to_categorical(y_train, 10)
        y_test = np_utils.to_categorical(y_test, 10)
        
    return (x_train, y_train), (x_test, y_test)

In [168]:
(x_train, y_train), (x_test, y_test) = prepare_data()

In [29]:
# (ds_train, ds_test), ds_info = tfds.load("mnist",
#                                          split=['train', 'test'], 
#                                          shuffle_files=True,  # размешает файлы
#                                          as_supervised=True,  # вернет tuple, иначе dict
#                                          with_info=True)  # вернет инфоррмацию о датасете

In [30]:
def normalize_image(image, label):
    return tf.cast(image, tf.float32)/255.0, label

In [31]:
BATCH_SIZE = 128

In [32]:
# настройка тренировочного набора
ds_train = ds_train.map(normalize_image)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)

# настройка тестового набора
ds_test = ds_test.map(normalize_image)
ds_test = ds_test.cache()
ds_test = ds_test.shuffle(ds_info.splits['test'].num_examples)
ds_test = ds_test.batch(BATCH_SIZE)

In [34]:
def scheduler(epoch, lr):
    if epoch < 2:
        return lr
    else:
        return lr * 0.9

In [35]:
lr_callback = K.callbacks.LearningRateScheduler(scheduler, verbose=1)

save_callback = K.callbacks.ModelCheckpoint('tf_checkpoint/', 
                                         save_weights_only=True, 
                                         monitor='accuracy',
                                         save_best_only=False)

In [36]:
class CustomCallback(K.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('val_accuracy') > 0.9:
            print("можно останавливать обучение")
            self.model.stop_training = True
    

In [164]:
def grid_model(optimizer='adam', init='glorot_uniform'):
    # simple model
    model = K.Sequential([
        K.Input(shape=(28, 28, 1)),
        layers.Conv2D(16, kernel_size=(5, 5), activation='elu'),
        layers.Conv2D(32, kernel_size=(5, 5), activation='elu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation='elu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='sigmoid'),
        layers.Dense(10)
    ])
#     model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [292]:
def define_model(optimizer='adam', activation='relu', kernel_initializer='he_uniform'):
    model = K.models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation=activation, 
                            kernel_initializer=kernel_initializer, 
                            input_shape=(28, 28, 1)))
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Flatten())
    
    model.add(layers.Dense(100, activation=activation, 
                           kernel_initializer=kernel_initializer))
    
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [186]:
model = define_model()

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=5)

Epoch 1/5
469/469 [==============================] - 15s 32ms/step - loss: 0.2481 - accuracy: 0.9260
Epoch 2/5
469/469 [==============================] - 15s 32ms/step - loss: 0.1011 - accuracy: 0.9710
Epoch 3/5
469/469 [==============================] - 15s 32ms/step - loss: 0.0696 - accuracy: 0.9799
Epoch 4/5
469/469 [==============================] - 16s 33ms/step - loss: 0.0533 - accuracy: 0.9842
Epoch 5/5
469/469 [==============================] - 16s 34ms/step - loss: 0.0436 - accuracy: 0.9875


In [293]:
keras_clf = KerasClassifier(build_fn=define_model, batch_size=128)

In [294]:
# определим те гиперпараметры, которые будем тюнить
optimizers = ["Adam", "Nadam", "Adadelta", "RMSprop"]
weight_init = ["he_uniform", "glorot_uniform", "normal", "uniform"]
learning_rate_cv = np.array([1e-1, 1e-2, 1e-3])

# создадим сетку из этих гиперпараметров
param_grid = dict(optimizer=optimizers, kernel_initializer=weight_init)

In [295]:
param_grid

{'optimizer': ['Adam', 'Nadam', 'Adadelta', 'RMSprop'],
 'kernel_initializer': ['he_uniform', 'glorot_uniform', 'normal', 'uniform']}

In [296]:
# создание класса GridSearchCV
validator = GridSearchCV(estimator=keras_clf, param_grid=param_grid)

In [299]:
result = validator.fit(x_train, y_train, epochs=2)

Epoch 1/2
313/313 [==============================] - 11s 34ms/step - loss: 0.2368 - accuracy: 0.9285
Epoch 2/2
157/157 [==============================] - 2s 10ms/step - loss: 0.0751 - accuracy: 0.9775
Epoch 1/2
313/313 [==============================] - 11s 34ms/step - loss: 0.2611 - accuracy: 0.9226
Epoch 2/2
157/157 [==============================] - 2s 11ms/step - loss: 0.0797 - accuracy: 0.9762
Epoch 1/2
313/313 [==============================] - 11s 34ms/step - loss: 0.1939 - accuracy: 0.9424
Epoch 2/2
157/157 [==============================] - 2s 11ms/step - loss: 0.0694 - accuracy: 0.9785
Epoch 1/2
313/313 [==============================] - 12s 39ms/step - loss: 0.2075 - accuracy: 0.9393
Epoch 2/2
157/157 [==============================] - 2s 10ms/step - loss: 0.0657 - accuracy: 0.9794
Epoch 1/2
313/313 [==============================] - 12s 39ms/step - loss: 0.2351 - accuracy: 0.9284
Epoch 2/2
157/157 [==============================] - 2s 10ms/step - loss: 0.0748 - accuracy: 0.

In [305]:
print(f'best accuracy is {result.best_score_:.3f}')

best accuracy is 0.977


In [307]:
result.best_params_

{'kernel_initializer': 'he_uniform', 'optimizer': 'Adam'}

In [315]:
cvres = result.cv_results_

In [328]:
for mean_test_score, params in zip(cvres["mean_test_score"], cvres['params']):
    print(f'{mean_test_score:.3f}', params)

0.977 {'kernel_initializer': 'he_uniform', 'optimizer': 'Adam'}
0.977 {'kernel_initializer': 'he_uniform', 'optimizer': 'Nadam'}
0.486 {'kernel_initializer': 'he_uniform', 'optimizer': 'Adadelta'}
0.974 {'kernel_initializer': 'he_uniform', 'optimizer': 'RMSprop'}
0.973 {'kernel_initializer': 'glorot_uniform', 'optimizer': 'Adam'}
0.975 {'kernel_initializer': 'glorot_uniform', 'optimizer': 'Nadam'}
0.319 {'kernel_initializer': 'glorot_uniform', 'optimizer': 'Adadelta'}
0.974 {'kernel_initializer': 'glorot_uniform', 'optimizer': 'RMSprop'}
0.975 {'kernel_initializer': 'normal', 'optimizer': 'Adam'}
0.972 {'kernel_initializer': 'normal', 'optimizer': 'Nadam'}
0.136 {'kernel_initializer': 'normal', 'optimizer': 'Adadelta'}
0.973 {'kernel_initializer': 'normal', 'optimizer': 'RMSprop'}
0.973 {'kernel_initializer': 'uniform', 'optimizer': 'Adam'}
0.966 {'kernel_initializer': 'uniform', 'optimizer': 'Nadam'}
0.144 {'kernel_initializer': 'uniform', 'optimizer': 'Adadelta'}
0.971 {'kernel_initi

In [338]:
""" Let's try search by random approach """

" Let's try search by random approach "

In [335]:
%%time
# создание класса RandomSearchCV
validator_rnd = RandomizedSearchCV(estimator=keras_clf, param_distributions=param_grid, random_state=42)

result = validator_rnd.fit(x_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


469/469 [==============================] - 16s 34ms/step - loss: 0.1814 - accuracy: 0.9452


In [336]:
result.best_params_

{'optimizer': 'Adam', 'kernel_initializer': 'he_uniform'}

In [337]:
cvres = result.cv_results_

for score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(f'{score:.3f}', params)

0.968 {'optimizer': 'Adam', 'kernel_initializer': 'he_uniform'}
0.950 {'optimizer': 'Nadam', 'kernel_initializer': 'he_uniform'}
0.959 {'optimizer': 'Nadam', 'kernel_initializer': 'glorot_uniform'}
0.133 {'optimizer': 'Adadelta', 'kernel_initializer': 'uniform'}
0.955 {'optimizer': 'Nadam', 'kernel_initializer': 'uniform'}
0.960 {'optimizer': 'RMSprop', 'kernel_initializer': 'normal'}
0.962 {'optimizer': 'Adam', 'kernel_initializer': 'normal'}
0.949 {'optimizer': 'Nadam', 'kernel_initializer': 'normal'}
0.281 {'optimizer': 'Adadelta', 'kernel_initializer': 'he_uniform'}
0.945 {'optimizer': 'RMSprop', 'kernel_initializer': 'uniform'}
